## Building Chris's Protocol deconfliction

ToDos:
- Implementation of Protocol based w/ round-robin
    - includes backpressure math, not cycling detection
    - random time based agent starting

- Visualizations

In [1]:
from __future__ import division
from __future__ import print_function
import collections
import math
import numpy as np

_Hex = collections.namedtuple("Hex", ["q", "r", "s"])
class _Hex(_Hex):
    def __repr__(self):
        return f'h({self.q}, {self.r}, {self.s})'

def Hex(q, r, s):
    assert not (round(q + r + s) != 0), "q + r + s must be 0"
    return _Hex(q, r, s)

def hex_add(a, b):
    return Hex(a.q + b.q, a.r + b.r, a.s + b.s)

def hex_subtract(a, b):
    return Hex(a.q - b.q, a.r - b.r, a.s - b.s)

def hex_scale(a, k):
    return Hex(a.q * k, a.r * k, a.s * k)

def hex_rotate_left(a):
    return Hex(-a.s, -a.q, -a.r)

def hex_rotate_right(a):
    return Hex(-a.r, -a.s, -a.q)

hex_directions = [Hex(1, 0, -1), Hex(1, -1, 0), Hex(0, -1, 1), Hex(-1, 0, 1), Hex(-1, 1, 0), Hex(0, 1, -1)]
def hex_direction(direction):
    return hex_directions[direction]

def hex_neighbor(hex, direction):
    return hex_add(hex, hex_direction(direction))

hex_diagonals = [Hex(2, -1, -1), Hex(1, -2, 1), Hex(-1, -1, 2), Hex(-2, 1, 1), Hex(-1, 2, -1), Hex(1, 1, -2)]
def hex_diagonal_neighbor(hex, direction):
    return hex_add(hex, hex_diagonals[direction])

def hex_length(hex):
    return (abs(hex.q) + abs(hex.r) + abs(hex.s)) // 2

def hex_distance(a, b):
    return hex_length(hex_subtract(a, b))

def hex_round(h):
    qi = int(round(h.q))
    ri = int(round(h.r))
    si = int(round(h.s))
    q_diff = abs(qi - h.q)
    r_diff = abs(ri - h.r)
    s_diff = abs(si - h.s)
    if q_diff > r_diff and q_diff > s_diff:
        qi = -ri - si
    else:
        if r_diff > s_diff:
            ri = -qi - si
        else:
            si = -qi - ri
    return Hex(qi, ri, si)

def hex_lerp(a, b, t):
    return Hex(a.q * (1.0 - t) + b.q * t, a.r * (1.0 - t) + b.r * t, a.s * (1.0 - t) + b.s * t)

def hex_linedraw(a, b):
    N = hex_distance(a, b)
    a_nudge = Hex(a.q + 1e-06, a.r + 1e-06, a.s - 2e-06)
    b_nudge = Hex(b.q + 1e-06, b.r + 1e-06, b.s - 2e-06)
    results = []
    step = 1.0 / max(N, 1)
    for i in range(0, N + 1):
        results.append(hex_round(hex_lerp(a_nudge, b_nudge, step * i)))
    return results

def create_hex_grid(radius):
    """
    Inputs:
        radius: radius of hex grid
    Outputs:
        coords: set of tuples (q,r,s) hex coords
    """
    coords = set()
    for q in range(-radius, radius+1):
        r1 = max(-radius, -q - radius)
        r2 = min(radius, -q + radius)

        for r in range(r1, r2+1):
            coords.add(Hex(q, r, -q-r))
    return coords

In [2]:
class Grid():
    """
    Contains the environment, the auctioneer
    
    Grid is small grid of 6 hexagons surrounding 7th center (q, r, s)
    
    
        (0, -1, 1)   (1, -1, 0)
    (-1, 0, 1)  (0,0,0)  (1, 0, -1)
        (-1, 1, 0)   (0, 1, -1)
    """
    
    
    def __init__(self, radius=1):
        """
        Creates Grid object
        
        Inputs:
            agents: list, agents in the environment
        Returns:
        
        """
        self._revenue = 0
        self.coords = create_hex_grid(radius)
        
    @property
    def revenue(self):
        return self._revenue
        
    def step_sim(self, bids):
        """
        Inputs:
            bids: list of tuples (loc, price)
        Returns:
            output: list of tuples (next loc, winning bid) of size (# agents)
        """
        
        requests = {}
        for i, (loc, price) in enumerate(bids):
            if loc is None: continue
            if price == -1: continue
            
            
            assert loc in self.coords
            if loc not in requests: requests[loc] = []
            requests[loc].append((i, price))
        
        output = self.sealed_second_price(requests, len(bids))
        return output
        
    def sealed_second_price(self, requests, num_agents):
        """
        Sealed second price auction for locations
    
        Inputs:
            requests: dictionary of (loc, (agent_id, bid)) to resolve
            num_agents: number of agents bidding
        
        Returns:
            output: list of tuples (next loc, winning bid) of size (# agents)
        """
        output = [(None, 0) for i in range(num_agents)]
        revenue = 0
        for loc, bids in requests.items():
            print("Location, Bid (agent, bid):", loc, bids)
            winner = max(bids, key=lambda x: x[1])    # Note: if ties, then lower ID # wins (first in line)
            bids.remove(winner)
            
            if not bids: price = (None, 0)
            else: price = max(bids, key=lambda x: x[1])
            
            output[winner[0]] = (loc, price[1])
            revenue += price[1]
        
        # track system revenue earned
        self._revenue += revenue
        
        return output
        
        # get all agent requests and bids
        # requests in dictionary with loc as key and (agent, bid) as value
        
        # for every request
            # find highest bid, agent gets to move
            # agent payment increments, revenue of system increments
        # all agents pay their variable costs for existing
        
        # returns a list of [0, 1] where 0 means that agent holds, 1 is that agent moves
        # OR
        # returns a list of [(None, price), (loc, price)] where None means hold, (loc) indicates move; 
        # price is price paid to grid <- do this
        
        # def check_request(self, ) #agent request checker, subfunction for moving stuff 

In [7]:
class Agent():
    
    def __init__(self, origin, dest, var_cost):
        
        """
        Agent navigating the environment
        
        Input:
            origin: Hex, initial location
            dest: Hex, destination location
            var_cost: double, variable cost of operation
        Returns:
        
        """

        self._loc = origin
        self._var_cost = var_cost

        self._steps = hex_linedraw(origin, dest)[1:]
        self._pay_costs = 0
        self._wait_costs = 0
        
        self._origin = origin
        self._dest = dest
        
    @property
    def bid(self):
        """
        Send out bid of (loc, price)
        Inputs:
        Returns:
            out: tuple of (loc, price)
        """
        if not self._steps: return (None, -1)  # if you're done
        
        # check that requested location is right next to current
#         dist = sum(abs(self._steps[0][i] - self._loc[i]) for i in range(3)) / 2
        assert hex_distance(self._steps[0], self._loc) == 1
        
        return (self._steps[0], self._var_cost)
    
    @property
    def loc(self):
        """
        Return current location
        """
        return self._loc
    
    @property
    def costs(self):
        """
        Return costs incurred by agent so far
        This is essentially the extra costs incurred on top of costs from operatin w/o anyone else
        """
        return (self._pay_costs, self._wait_costs)
    
    def move(self, command):
        """
        Move the bot
        Inputs:
            command: tuple of (next_loc, bid)
        Outputs:
            out: boolean to indicate at goal already
        """
        if not self._steps: return True  # if you're at the goal already
        
        next_loc, bid = command
        # if command is None, not cleared to move and you eat the variable cost
        if next_loc == None:
            self._wait_costs += self._var_cost
            return False
        
        # else move
        assert next_loc == self._steps[0]
        del self._steps[0]
        self._loc = next_loc
        
        # pay the bid price
        self._pay_costs += bid
        return False

In [8]:
def agents_3_test_grid(radius = 1, iters = 10):
    """
    A standard test of 3 agents going across each other meeting in the middle
    """
    agent1 = Agent(Hex(0, -radius, radius), Hex(0, radius, -radius), 1)
    agent2 = Agent(Hex(-radius, 0, radius), Hex(radius, 0, -radius), 1)
    agent3 = Agent(Hex(-radius, radius, 0), Hex(radius, -radius, 0), 1)
    
    agents = [agent1, agent2, agent3]
    grid = Grid(radius)
    
    for k in range(iters):
        print(f"Cycle {k}")

        bids = [el.bid for el in agents]
        commands = grid.step_sim(bids)
        print("Commands:", commands)

        for i, command in enumerate(commands):
            agents[i].move(command)

        print("Total Revenue: ", grid.revenue)
        print("\n")

    for i, agent in enumerate(agents):
        print("Agent ", i, " costed ", agent.costs)   # think of this as extra/delayed costs
    print("Total Revenue: ", grid.revenue)

agents_3_test_grid(radius = 3)

Cycle 0
Location, Bid (agent, bid): h(0, -2, 2) [(0, 1)]
Location, Bid (agent, bid): h(-2, 0, 2) [(1, 1)]
Location, Bid (agent, bid): h(-2, 2, 0) [(2, 1)]
Commands: [(h(0, -2, 2), 0), (h(-2, 0, 2), 0), (h(-2, 2, 0), 0)]
Total Revenue:  0


Cycle 1
Location, Bid (agent, bid): h(0, -1, 1) [(0, 1)]
Location, Bid (agent, bid): h(-1, 0, 1) [(1, 1)]
Location, Bid (agent, bid): h(-1, 1, 0) [(2, 1)]
Commands: [(h(0, -1, 1), 0), (h(-1, 0, 1), 0), (h(-1, 1, 0), 0)]
Total Revenue:  0


Cycle 2
Location, Bid (agent, bid): h(0, 0, 0) [(0, 1), (1, 1), (2, 1)]
Commands: [(h(0, 0, 0), 1), (None, 0), (None, 0)]
Total Revenue:  1


Cycle 3
Location, Bid (agent, bid): h(0, 1, -1) [(0, 1)]
Location, Bid (agent, bid): h(0, 0, 0) [(1, 1), (2, 1)]
Commands: [(h(0, 1, -1), 0), (h(0, 0, 0), 1), (None, 0)]
Total Revenue:  2


Cycle 4
Location, Bid (agent, bid): h(0, 2, -2) [(0, 1)]
Location, Bid (agent, bid): h(1, 0, -1) [(1, 1)]
Location, Bid (agent, bid): h(0, 0, 0) [(2, 1)]
Commands: [(h(0, 2, -2), 0), (h(1,

In [ ]:
def agent_test_random(radius = 1, iters = 10, seed = 0):
    """
    Set of random connections on a grid for data for stuffs later
    
    """
    np.random.seed(seed)
    
    # Agents get random OD and random departure time (arrival time judged by steps to there)

### Trying more complex things

- grid cleanup - checking for legal moves
    - create larger grids automagically
    - create random agent paths
    - what were the agent metrics you wanted again? paid, cost, utility?
- try this on larger scale problems - >3 agents, with >3 steps and targets - but WITHOUT cycles or backpressure yet
- Integrate into Chris's setup of cycles and such 

